In [1]:
# playing with pandas

In [3]:
import math
import random
from functools import partial
from typing import List, Dict, Optional
import numpy as np

from api_wafermap.crud.crud_param import get_parameter_by_name
from api_wafermap.forms.form_utils import (
    convert_input_from_ranged_to_single_value,
    check_params_range,
)

from numba import njit

"""
@njit
def uniform_noise(point_coord_value: float, noise_value: float):
    return point_coord_value + noise_value * rd.uniform(0,1)


@njit
def normal_noise(point_coord_value: float, sigma: float, sign: Optional[bool]):
    mean = 0
    noise_value = np.random.normal(mean, sigma)
    if not sign :
        return point_coord_value - np.abs(noise_value)
    if sign is None:
        return point_coord_value + noise_value
    
    return point_coord_value + np.abs(noise_value)


@njit
def apply_noise_to_form(
    point_coord_value: float,
    noise_value: float,
    sigma: float = 0,
    sign: Optional[bool] = None,
):

    point_coord_value = uniform_noise(point_coord_value, noise_value)
    point_coord_value = normal_noise(point_coord_value, sigma, sign)

    return point_coord_value
"""

@njit
def uniform_noise(point_coord_value: float, noise_value: float):
    return point_coord_value + noise_value * random.uniform(0, 1)


@njit
def normal_noise(point_coord_value: float, sigma: float, sign: Optional[bool]):
    mean = 0
    noise_value = np.random.normal(mean, sigma)
    if sign is None:
        point_coord_value += noise_value
    elif sign:
        point_coord_value += np.abs(noise_value)
    else:
        point_coord_value -= np.abs(noise_value)

    return point_coord_value


@njit
def apply_noise_to_form(
    point_coord_value: float,
    noise_value: float,
    sigma: float = 0,
    sign: Optional[bool] = None,
):

    point_coord_value = uniform_noise(point_coord_value, noise_value)

    if sigma != 0:
        point_coord_value = normal_noise(point_coord_value, sigma, sign)

    return point_coord_value



def create_circle_points(
    num_points_minimum: int = 10000,
    num_points_maximum: int = 10000,
    radius_minimum: float = 10,
    radius_maximum: float = 10,
    x_center: float = 200,
    y_center: float = 200,
    noise_x_minimum: float = 0,
    noise_x_maximum: float = 0,
    noise_y_minimum: float = 0,
    noise_y_maximum: float = 0,
    sigma_x_minimum: float = 0,
    sigma_x_maximum: float = 0,
    sigma_y_minimum: float = 0,
    sigma_y_maximum: float = 0,
    sign_x: Optional[bool] = None,
    sign_y: Optional[bool] = None,
    angle_limit_minimum: float = 360,
    angle_limit_maximum: float = 360,
) -> List[Dict]:
    list_variable_ranged = [
        (num_points_minimum, num_points_maximum),
        (radius_minimum, radius_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
        (sigma_x_minimum, sigma_x_maximum),
        (sigma_y_minimum, sigma_y_maximum),
    ]

    check_params_range(list_variable_ranged)

    [
        num_points,
        radius,
        noise_x,
        noise_y,
        sigma_x,
        sigma_y,
    ] = convert_input_from_ranged_to_single_value(
        (num_points_minimum, num_points_maximum),
        (radius_minimum, radius_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
        (sigma_x_minimum, sigma_x_maximum),
        (sigma_y_minimum, sigma_y_maximum),
    )

    
    results = create_circle_points_jitted(angle_limit_minimum, 
                                       angle_limit_maximum,
                                        radius,
                                        x_center,
                                        y_center,
                                        noise_x,
                                        noise_y,
                                        sigma_x,
                                        sigma_y,
                                        sign_x,
                                        sign_y,
                                        num_points,
                                      )

    #return [{"x": x[0], "y":x[1]} for x in results]
    

@njit
def create_circle_points_jitted(angle_limit_minimum, 
                                angle_limit_maximum,
                                radius,
                                x_center,
                                y_center,
                                noise_x,
                                noise_y,
                                sigma_x,
                                sigma_y,
                                sign_x,
                                sign_y,
                                num_points
                               ):
    angle_limit_minimum = math.radians(angle_limit_minimum)
    angle_limit_maximum = math.radians(angle_limit_maximum)
    random_circle_points = np.zeros((num_points, 2))
    
    for i in range(0, len(random_circle_points)):
        angle = rd.uniform(angle_limit_minimum, angle_limit_maximum)
        random_circle_points[i] =  _create_circle_point(
                                angle,
                                radius,
                                x_center,
                                y_center,
                                noise_x,
                                noise_y,
                                sigma_x,
                                sigma_y,
                                sign_x,
                                sign_y,
                            )
        
    
    return random_circle_points

@njit
def _create_circle_point(
    angle: float,
    radius: float,
    x_center: float = 200,
    y_center: float = 200,
    noise_x: float = 10,
    noise_y: float = 10,
    sigma_x: float = 10,
    sigma_y: float = 10,
    sign_x: bool = None,
    sign_y: bool = None,
    is_circle_empty: bool = True,
):
    multiplier: float = 1
    #if not is_circle_empty:
    #    multiplier = np.sqrt(np.random.rand())
    x = radius * multiplier * np.cos(angle) + x_center
    y = radius * multiplier * np.sin(angle) + y_center

    x = apply_noise_to_form(x, noise_x, sigma_x, sign_x)
    y = apply_noise_to_form(y, noise_y, sigma_y, sign_y)

    return np.array([x, y])


In [4]:
%%time
create_circle_points()

Wall time: 2.34 s


In [541]:
import math
import random as rd
from functools import partial
from typing import List, Dict, Optional
import numpy as np

from api_wafermap.crud.crud_param import get_parameter_by_name
from api_wafermap.forms.form_utils import (
    from_degrees_to_radians,
    convert_input_from_ranged_to_single_value,
    check_params_range,
)

from numba import njit

def uniform_noise(point_coord_value: float, noise_value: float):
    point_coord_value += noise_value * rd.uniform(0,1)
    return point_coord_value


def normal_noise(point_coord_value: float, sigma: float, sign: Optional[bool]):
    mean = 0
    noise_value = np.random.normal(mean, sigma)
    if sign is None:
        point_coord_value += noise_value
    elif sign:
        point_coord_value += np.abs(noise_value)
    else:
        point_coord_value -= np.abs(noise_value)

    return point_coord_value


def apply_noise_to_form(
    point_coord_value: float,
    noise_value: float,
    sigma: float = 0,
    sign: Optional[bool] = None,
):

    if noise_value != 0:
        point_coord_value = uniform_noise(point_coord_value, noise_value)

    if sigma != 0:
        point_coord_value = normal_noise(point_coord_value, sigma, sign)

    return point_coord_value




def create_circle_points(
    num_points_minimum: int = 10000,
    num_points_maximum: int = 10000,
    radius_minimum: float = 10,
    radius_maximum: float = 10,
    x_center: float = 200,
    y_center: float = 200,
    noise_x_minimum: float = 0,
    noise_x_maximum: float = 0,
    noise_y_minimum: float = 0,
    noise_y_maximum: float = 0,
    sigma_x_minimum: float = 0,
    sigma_x_maximum: float = 0,
    sigma_y_minimum: float = 0,
    sigma_y_maximum: float = 0,
    sign_x: Optional[bool] = None,
    sign_y: Optional[bool] = None,
    angle_limit_minimum: float = 360,
    angle_limit_maximum: float = 360,
) -> List[Dict]:
    list_variable_ranged = [
        (num_points_minimum, num_points_maximum),
        (radius_minimum, radius_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
        (sigma_x_minimum, sigma_x_maximum),
        (sigma_y_minimum, sigma_y_maximum),
    ]

    check_params_range(list_variable_ranged)

    [
        num_points,
        radius,
        noise_x,
        noise_y,
        sigma_x,
        sigma_y,
    ] = convert_input_from_ranged_to_single_value(
        (num_points_minimum, num_points_maximum),
        (radius_minimum, radius_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
        (sigma_x_minimum, sigma_x_maximum),
        (sigma_y_minimum, sigma_y_maximum),
    )

    angle_limit_minimum = from_degrees_to_radians(angle_limit_minimum)
    angle_limit_maximum = from_degrees_to_radians(angle_limit_maximum)

    angles = [
        rd.uniform(angle_limit_minimum, angle_limit_maximum)
        for _ in range(0, int(num_points))
    ]
    random_circle_points = [
        _create_circle_point(
            angle,
            radius,
            x_center,
            y_center,
            noise_x,
            noise_y,
            sigma_x,
            sigma_y,
            sign_x,
            sign_y,
        )
        for angle in angles
    ]
    return random_circle_points


def _create_circle_point(
    angle: float,
    radius: float,
    x_center: float = 200,
    y_center: float = 200,
    noise_x: float = 0,
    noise_y: float = 0,
    sigma_x: float = 0,
    sigma_y: float = 0,
    sign_x: bool = None,
    sign_y: bool = None,
    is_circle_empty: bool = True,
) -> Dict:
    multiplier: float = 1
    if not is_circle_empty:
        multiplier = math.sqrt(rd.random())
    x = radius * multiplier * math.cos(angle) + x_center
    y = radius * multiplier * math.sin(angle) + y_center

    x = apply_noise_to_form(x, noise_x, sigma_x, sign_x)
    y = apply_noise_to_form(y, noise_y, sigma_y, sign_y)

    return {"x": x, "y": y}


In [584]:
%%time
create_circle_points()

Wall time: 14 ms


[{'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 {'x': 210.0, 'y': 200.0},
 

In [198]:
import math
from functools import partial
from typing import List, Dict

from api_wafermap.crud.crud_param import get_parameter_by_name
from api_wafermap.forms.form_utils import (
    convert_input_from_ranged_to_single_value,
    apply_noise_to_form,
    check_params_range,
)
from api_wafermap.schemas.schema_form import SpiralFormOutput
from api_wafermap.schemas.schema_param import (
    ArcParamRange,
    SeparationParam,
    ArcParam,
    SeparationParamRange,
    NumPointsParam,
    NumPointsParamRange,
    NoiseXParam,
    NoiseXRange,
    NoiseYParam,
    NoiseYRange,
    OrientationParam,
    OrientationParamRange,
)


def create_spiral_points(
    num_points_minimum: int = 1000,
    num_points_maximum: int = 1000,
    arc_minimum: float = 0.8,
    arc_maximum: float = 0.8,
    separation_minimum: float = 0.6,
    separation_maximum: float = 0.6,
    x_center: float = 200,
    y_center: float = 200,
    orientation_minimum: float = 1.0,
    orientation_maximum: float = 1.0,
    noise_x_minimum: float = 0.0,
    noise_x_maximum: float = 0.0,
    noise_y_minimum: float = 0.0,
    noise_y_maximum: float = 0.0,
) -> List[Dict[str, float]]:
    list_variable_ranged = [
        (num_points_minimum, num_points_maximum),
        (arc_minimum, arc_maximum),
        (separation_minimum, separation_maximum),
        (orientation_minimum, orientation_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
    ]
    check_params_range(list_variable_ranged)
    [
        num_points,
        arc,
        separation,
        orientation,
        noise_x,
        noise_y,
    ] = convert_input_from_ranged_to_single_value(
        (num_points_minimum, num_points_maximum),
        (arc_minimum, arc_maximum),
        (separation_minimum, separation_maximum),
        (orientation_minimum, orientation_maximum),
        (noise_x_minimum, noise_x_maximum),
        (noise_y_minimum, noise_y_maximum),
    )


    list_points_spiral = create_spiral_points_jitted(
        int(num_points),
        arc,
        separation,
        orientation,
        noise_x,
        noise_y,
        x_center,
        y_center,
    )
    
    
    return  [{"x": x[0], "y": x[1]} for x in list_points_spiral]

@njit
def create_spiral_points_jitted(
        num_points,
        arc,
        separation,
        orientation,
        noise_x,
        noise_y,
    x_center,
    y_center,
):
    list_points_spiral = np.zeros((num_points,2))

    po = 0
    for point_x, point_y in _spiral_points_generator(
        x_center, y_center, arc, separation, orientation
    ):

        point_x = float(apply_noise_to_form(point_x, noise_x))
        point_y = float(apply_noise_to_form(point_y, noise_y))

        #if po < num_points:
        #    list_points_spiral.append({"x": point[0], "y": point[1]})
        #    po = po + 1
        
        list_points_spiral[po] = np.array([point_x, point_y])
        
        po = po+1
        if po > num_points:
            break

    return list_points_spiral

@njit
def _spiral_points_generator(
    x_center: float = 200,
    y_center: float = 200,
    arc: float = 1,
    separation: float = 1,
    orientation: float = 1,
):
    """
    generate points on an Archimedes' spiral with `arc` giving the length of arc between two points
    and `separation` giving the distance between consecutive turnings
    """



    yield x_center, y_center

    rad = arc
    dist_rad = separation / (2 * math.pi)
    phi = rad / dist_rad
    while True:
        yield p2c(rad, phi, orientation)
        phi += arc / rad
        rad = dist_rad * phi

        
@njit
def p2c(radius: float, angle: float, orientation):
    """
    polar to cartesian
    """
    return (
        radius * orientation * math.cos(angle),
        radius * orientation * math.sin(angle),
    )

In [213]:
%%time
points = create_spiral_points()

Wall time: 2 ms


In [214]:
points

[{'x': 200.0, 'y': 200.0},
 {'x': -0.40000000000000063, 'y': 0.6928203230275507},
 {'x': -0.8944957469705771, 'y': 0.04224938517398055},
 {'x': -0.6501343146705301, 'y': -0.734370166236037},
 {'x': 0.09623672404720128, 'y': -1.0543093481473471},
 {'x': 0.8471013214143889, 'y': -0.7491627003781982},
 {'x': 1.1982394725035561, 'y': -0.01999883493908304},
 {'x': 1.004118777814871, 'y': 0.7647062480325807},
 {'x': 0.3732699808748139, 'y': 1.2689176308702486},
 {'x': -0.4325859527239206, 'y': 1.3109067367357838},
 {'x': -1.1231333106997567, 'y': 0.8944439526031596},
 {'x': -1.479092013269981, 'y': 0.17136228767930176},
 {'x': -1.4052047494344457, 'y': -0.630792290152297},
 {'x': -0.936004113801147, 'y': -1.2851631183977035},
 {'x': -0.20631932233625322, 'y': -1.6248931776200553},
 {'x': 0.5967756577217139, 'y': -1.5753312035278308},
 {'x': 1.2847287815822057, 'y': -1.1585019812725692},
 {'x': 1.7091833537374135, 'y': -0.4754850501452152},
 {'x': 1.7879216112809349, 'y': 0.32461221676662616}